In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

# PCA

In [ ]:
# expr is a cell x gene matrix

In [ ]:
n_components = 100
pca_op = decomposition.PCA(n_components = n_components, random_state=seed)
data_pca = pca_op.fit_transform(expr)

# format
data_pca = pd.DataFrame(data_pca, 
            columns = ['PC_' + str(i) for i in range(1,n_components+1)], 
            index = expr.index)

# # here is how explained variance is calculated - 
# # - analgous to seurat calculation and stored in pca_op.explained_variance_ratio_:

eigValues = pca_op.singular_values_**2
explained_variance_ = eigValues / (n_samples - 1)
total_var = np.var(expr, ddof=1, axis=0).sum() # variance of genes total
explained_variance_ratio_ = explained_variance_ / total_var

In [ ]:
# dataframe of explained variance
thresh = 0.001 #(0.1% of variance explained)
ev = pd.DataFrame(data = {'Explained_Variance': pca_op.explained_variance_ratio_, 
                         'Cumulative_Explained_Variance': np.cumsum(pca_op.explained_variance_ratio_)})
ev['Difference'] = ev.Explained_Variance.diff().abs().tolist()
n_pcs = n_components # because n_pcs is 7, and 200 components explains only 38% of total variance

print('The total PCs (at which additional variance explained is < 0.1%) used is: '.format(n_pcs))

# visualize
ev = ev.iloc[:n_components, ]
ev['PC'] = range(1, n_components + 1)
fig, ax = plt.subplots(figsize = (5,5))
sns.scatterplot(data = ev, x = 'PC', y = 'Cumulative_Explained_Variance')

# UMAP

In [ ]:
import umap # umap-learn

umap_op = umap.UMAP(n_components = 2, random_state = seed)
data_umap = umap_op.fit_transform(data_pca[:n_pcs])
data_umap = pd.DataFrame(data_umap, 
            columns = ['UMAP_' + str(i) for i in range(1,2+1)], 
            index = expr.index)

# NMI

In [ ]:
# there is also a simple sklearn.metrics.normalized_mutual_info_score, but I've hadd issues with it 

from scipy.stats import entropy
from sklearn.metrics import mutual_info_score as MI

def entropy1(x): # double check this is correct
    value,counts = np.unique(x, return_counts=True)
    return entropy(counts, base=None)

def NMI(x,y):
    return MI(x,y)/(entropy1(x) + entropy1(y))